# Anomaly detection in GPU data

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
import time
%load_ext autoreload
%autoreload 2

## Data processing

In [2]:
with open('functimes_combined.dat', "r") as f:
    raw_data = [x.strip("\n").split() for x in f.readlines()]

run_time = {}
for i in raw_data:
    func_name = []
    for j in range(len(i)):
        if not i[j].isdigit(): # get the func name
            func_name.append(i[j])
        else:
            runtime = [int(x) for x in i[j:]]
            key = "_".join(func_name).strip('\"')
            # if a key already exists, then the new runtime list will append to the existing list
            value = run_time.get(key)
            if value == None:
                run_time[key] = runtime
            else:
                run_time[key] = value.append(runtime)
    
            break # exit the loop once we find the position of the first numeric element

We see that the length of` (run_time)` isn't equal to the raw data

In [ ]:
len(run_time), len(raw_data)

We can collect the function name and check if anything is wrong:

In [ ]:
fun_name = []

for i in raw_data:
    func_name = []
    for j in range(len(i)):
        if not i[j].isdigit(): 
            func_name.append(i[j]) # get the func name
    key = "_".join(func_name).strip('\"')
    fun_name.append(key)

In [ ]:
for name, i in zip(fun_name, range(len(fun_name))):
    try:
        run_time[name]
    except:
        print(name, i)

If we check the original data file we will notice this is really the case. No runtime data follows `TOTAL`. And their is a blank entry in line 354.

Next let's select the most frequently called functions:

In [ ]:
len_list = []
for key, value in run_time.items():
    len_list.append(len(run_time[key]))

df_len = pd.DataFrame(len_list, index=run_time.keys())
df_len = df_len.reset_index()
df_len = df_len.rename(columns={"index": 'func_name', 0:'call_times'})
df_len = df_len.sort_values('call_times', ascending = False)

In [ ]:
df_freq = df_len[df_len['call_times'] > 10000]
subset = df_freq.sample(frac = 0.5, random_state = 1)

In [ ]:
def sample_portion(dic, func, frac = 0.3):    
    func_runtime = dic[func]
    # total length of the sample
    length = int(len(func_runtime) * frac)
    # select where to start taking the data
    start_position = np.random.randint(0, (len(func_runtime) - length))
    sample_runtime = func_runtime[start_position : start_position + length]
    df_sample = pd.DataFrame({'run_time' : sample_runtime})
    df_sample.name = func
    
    return df_sample   

In [ ]:
i = np.random.randint(len(subset))
func_name = subset.iloc[i].func_name
sample = sample_portion(run_time, func_name)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats = ['retina']
sns.set_style("darkgrid")
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['figure.figsize'] = [13, 5.]
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['legend.fontsize'] = 17
plt.rcParams['legend.frameon'] = True

In [ ]:
def plot_result(outlier, normal, method, *args):
    total_points = len(outlier) + len(normal)
    ratio = len(outlier) / total_points
    para = args
    plt.figure()
    if ratio >= 0.01:
        plt.title("{}, outlier_ratio={:.2f}".format(method, ratio))
    else:
        plt.title("{}, {} outliers in {} points".format(method, len(outlier), total_points))
    g = plt.scatter(normal.index,  normal['run_time'], c='green', s=15, edgecolor='k')
    h = plt.scatter(outlier.index, outlier['run_time'], c='red', s=55, edgecolor='k')
    #plt.xlabel('timestamp', fontsize=15)
    plt.ylabel('run_time', fontsize=15)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.axis('tight')
    plt.legend([g, h],["Normal","Outlier"], prop={'size': 15})
    plt.show()

## Moving Gaussian

In [ ]:
def moving_gaussian(df, window_frac = 0.2, c = 6, plot = True):
    window = int(window_frac * len(df))
    mean = df.run_time.rolling(window = window).mean()
    mean.iloc[:(window-1)] = mean.iloc[window]
    std = df.run_time.rolling(window = window).std()
    std.iloc[:(window-1)] = std.iloc[window]

    upper = mean + c * std
    lower = mean - c * std
    a1 = df.run_time > upper
    a2 = df.run_time < lower
    
    anomalies = np.logical_or(a1, a2)
    normal = np.logical_not(anomalies)
    outlier = df[anomalies]
    normal = df[normal]
    
    if plot:
        plot_result(outlier, normal, 'Gaussian')    
    return outlier, normal

In [ ]:
moving_gaussian(sample, plot=True);

In [ ]:
start = time.time()
for i in range(10): # test 10 random samples
    np.random.seed(i)
    j = np.random.randint(len(subset))
    func_name = subset.iloc[j].func_name
    #print(j, func_name)
    sample = sample_portion(run_time, func_name)
    moving_gaussian(sample, plot=True);
end = time.time()
print("total time used: {}".format(end - start))

## Moving Quantile Range

In [ ]:
def moving_quantile(df, window_frac = 0.15, c = 6, plot=True):
    window = int(window_frac * len(df))
    q1 = df.run_time.rolling(window = window).quantile(0.1)
    q3 = df.run_time.rolling(window = window).quantile(0.9)
    q1.iloc[:(window-1)] = q1.iloc[window]
    q3.iloc[:(window-1)] = q3.iloc[window]

    upper = q3 + c * (q3 - q1)
    lower = q1 - c * (q3 - q1)
    a1 = df.run_time > upper
    a2 = df.run_time < lower
    
    anomalies = np.logical_or(a1, a2)
    normal = np.logical_not(anomalies)
    outlier = df[anomalies]
    normal = df[normal]
    
    if plot:
        plot_result(outlier, normal, 'Tukey')        
    return outlier, normal

In [ ]:
start = time.time()
for i in range(10): # test 10 random samples
    np.random.seed(i)
    j = np.random.randint(len(subset))
    func_name = subset.iloc[j].func_name
    sample = sample_portion(run_time, func_name)
    moving_quantile(sample, plot=True);
end = time.time()
print("total time used: {}".format(end - start))

## MinClusterDetector

`MinClusterDetector` treats multivariate time series as independent points in a high-dimensional space, divides them into clusters, and identifies values in the smallest cluster as anomalous. This may help capturing outliers in high-dimensional space.

In [ ]:
from adtk.data import validate_series

def transform_adtk(df):
    tmp = pd.to_datetime(df.index.values, unit='D',
               origin=pd.Timestamp('2020-01-01'))
    idx = pd.Index(tmp, name='timestamp')
    
    data = {'run_time': df['run_time'].values}
    df_trans = pd.DataFrame(data=data, index=idx)
    df_trans.name = df.name
    return df_trans

sample_adtk = transform_adtk(sample)
validate_series(sample_adtk); # validate the dataframe conforms with adtk requirement

In [ ]:
from adtk.detector import MinClusterDetector
from sklearn.cluster import KMeans

def min_cluster(df, n_clusters = 2, plot=True):
    min_cluster_detector = MinClusterDetector(KMeans(n_clusters = n_clusters))
    df = transform_adtk(df)
    anomalies = min_cluster_detector.fit_detect(df)
    normal = df[~anomalies]
    outlier = df[anomalies]
    
    if plot:
        plot_result(outlier, normal, 'Min Cluster')
    
    return outlier, normal

In [ ]:
start = time.time()
for i in range(10): # test 10 random samples
    np.random.seed(i)
    j = np.random.randint(len(subset))
    func_name = subset.iloc[j].func_name
    sample = sample_portion(run_time, func_name)
    min_cluster(sample, plot=True);
end = time.time()
print("total time used: {}".format(end - start))

## Relative Entropy Detector

In [ ]:
# from base import NAB_Dataset
from relative_entropy_detector import RelativeEntropyDetector

In [ ]:
def relative_entropy(df, window_frac = 0.01, bins = 40, plot = True):
    re_ad = RelativeEntropyDetector(df)
    window = int(window_frac * len(df))
    re_ad.W = window
    re_ad.N_bins = bins
    result = re_ad.run();
    outlier = result[result.anomaly_score > 0]
    normal = result[result.anomaly_score == 0]
    if plot:
        plot_result(outlier, normal, "RE")
    return outlier, normal

In [ ]:
start = time.time()
for i in range(10): # test 10 random samples
    np.random.seed(i)
    j = np.random.randint(len(subset))
    func_name = subset.iloc[j].func_name
    sample = sample_portion(run_time, func_name)
    relative_entropy(sample, plot=True);
end = time.time()
print("total time used: {}".format(end - start))

## KNN-CAD

A short introduction of KNN-CAD is [here](https://github.com/empyriumz/NAB/tree/master/nab/detectors/knncad).

In [ ]:
from knncad_detector import KnncadDetector

In [ ]:
knn = KnncadDetector(sample, 0.1)

In [ ]:
sample.iloc[:2000]

In [ ]:
def knn_cad(df, k = 8, dim = 2, plot=True):
    knn = KnncadDetector(df, 0.02)
    knn.dim = dim
    knn.k = k
    result = knn.run()
    out = result[result[1] > result[1].quantile(0.99)]
    normal = result[result[1] <= result[1].quantile(0.99)]
    out = out.rename(columns={0: 'run_time'})
    normal = normal.rename(columns={0: 'run_time'})
    if plot:
        plot_result(out, normal, 'knn')
    return out, normal

In [ ]:
start = time.time()
for i in range(10): # test 10 random samples
    np.random.seed(i)
    j = np.random.randint(len(subset))
    func_name = subset.iloc[j].func_name
    sample = sample_portion(run_time, func_name)
    knn_cad(sample.iloc[:int(0.3*len(sample))], plot=True);
end = time.time()
print("total time used: {}".format(end - start))

## Elliptic Envelope

In [ ]:
from sklearn.covariance import EllipticEnvelope

In [ ]:
EllipticEnvelope(contamination=0.01)

## PyOD

### CBLOF

In [ ]:
from pyod.models.cblof import CBLOF

def cblof(df, n_clusters=8, contamination=0.01, cut_off = 15, plot=True):
    x_ = df.run_time.values.reshape(-1, 1)
    high, low = x_.max(), np.quantile(x_, q=0.05)
    # enforce cutoff rule
    if (high - low) / low <= cut_off:
        out, normal = pd.DataFrame({'run_time':[]}), df
        print("cutoff rule applied with cutoff={}".format(cut_off))
    else:    
        clf = CBLOF(n_clusters = n_clusters, contamination = contamination)
        clf.fit(x_)
        result = clf.predict(x_)
        tmp = pd.DataFrame({'anomaly_score': result, 'run_time': x_.reshape(-1)})
        out = tmp[(tmp.anomaly_score > 0)]
        normal = tmp[(tmp.anomaly_score == 0)]

    if plot:
        plot_result(out, normal, 'CB_LOF')
    return out, normal

In [ ]:
start = time.time()
for i in range(10): # test 10 random samples
    np.random.seed(71*i+1)
    j = np.random.randint(len(subset))
    func_name = subset.iloc[j].func_name
    sample = sample_portion(run_time, func_name, frac=0.5)
    cblof(sample, plot=True);
end = time.time()
print("total time used: {}".format(end - start))

Let's benchmark with the traditional `LOF`

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

def lof(df, contamination = 0.01, n_neighbors = 100, plot = True):
    clf = LocalOutlierFactor(n_neighbors=n_neighbors, # broader estimation 
                             algorithm="auto", leaf_size=30, metric="minkowski", p=2, 
                             metric_params=None, contamination=contamination, n_jobs=-1)
    y_pred = clf.fit_predict(df.run_time.values.reshape(-1, 1))
    outlier = df[y_pred == -1]
    normal = df[y_pred == +1]
    if plot:
        plot_result(outlier, normal, 'lof')
    return outlier, normal

In [ ]:
start = time.time()
for i in range(10): # test 10 random samples
    np.random.seed(i)
    j = np.random.randint(len(subset))
    func_name = subset.iloc[j].func_name
    sample = sample_portion(run_time, func_name)
    lof(sample, plot=True);
end = time.time()
print("total time used: {}".format(end - start))

We see `CBLOF` is better and much faster.

### COPOD

In [ ]:
from pyod.models.copod import COPOD

def cop(df, contamination=0.01, cut_off = 20, plot=True):
    x_ = df.run_time.values.reshape(-1, 1)    
    # enforce cutoff rule
    highest, high, low = x_.max(), np.quantile(x_, q=0.99), np.quantile(x_, q=0.1)
    if (highest - low) / low <= cut_off:
        out, normal = pd.DataFrame({'run_time':[]}), df
        print("cutoff rule applied with cutoff={}".format(cut_off))
    else:
        if (high - low)/low < 0.5: # if majority of the data is narrowly distributed, lower the outlier ratio
            contamination = 0.1 * contamination
            print('contamination lowered with c={}'.format(contamination))
        clf = COPOD(contamination = contamination) 
        clf.fit(x_)
        result = clf.predict(x_)
        tmp = pd.DataFrame({'anomaly_score': result, 'run_time': x_.reshape(-1)})
        out = tmp[(tmp.anomaly_score > 0)]
        normal = tmp[(tmp.anomaly_score == 0)]
        
    if plot:
        plot_result(out, normal, 'COPOD')
    return out, normal

In [ ]:
start = time.time()
for i in range(10): # test 10 random samples
    seed = 78*i+31
    np.random.seed()
    j = np.random.randint(len(subset))
    func_name = subset.iloc[j].func_name
    sample = sample_portion(run_time, func_name, frac=0.5)
    print("seed = {}, func_name:{}".format(seed, func_name))
    cop(sample, plot=True);
end = time.time()
print("total time used: {}".format(end - start))

### Feature Bagging

In [ ]:
from pyod.models.feature_bagging import FeatureBagging

In [ ]:
feature_bag = FeatureBagging(contamination=0.01, max_features=1)

In [ ]:
x_.reshape(-1)

In [ ]:
feature_bag.fit(x_)

### HBOS

In [ ]:
from pyod.models.hbos import HBOS

def hbos(df, contamination=0.01, n_bins = 10, alpha = 0.2, cut_off = 20, plot=True):
    x_ = df.run_time.values.reshape(-1, 1)
    highest, high, low = x_.max(), np.quantile(x_, q=0.99), np.quantile(x_, q=0.1)
    # enforce cutoff rule
    if (highest - low) / low <= cut_off:
        out, normal = pd.DataFrame({'run_time':[]}), df
        print("cutoff rule applied with cutoff={}".format(cut_off))
    else:
        if (high - low) / low < 0.5: # if majority of the data is narrowly distributed, lower the outlier ratio
            contamination = 0.1 * contamination
            print('contamination lowered with c={}'.format(contamination))
        clf = HBOS(n_bins = n_bins, alpha = alpha, contamination = contamination) 
        clf.fit(x_)
        result = clf.predict(x_)
        tmp = pd.DataFrame({'anomaly_score': result, 'run_time': x_.reshape(-1)})
        out = tmp[(tmp.anomaly_score > 0)]
        normal = tmp[(tmp.anomaly_score == 0)]
        
    if plot:
        plot_result(out, normal, 'hbos')
    return out, normal

In [ ]:
start = time.time()
for i in range(10): # test 10 random samples
    seed = 1122*i+91
    np.random.seed(seed)
    j = np.random.randint(len(subset))
    func_name = subset.iloc[j].func_name
    print("seed={}".format(seed), func_name)
    #print()
    sample = sample_portion(run_time, func_name, frac=0.5)
    hbos(sample, plot=True);
end = time.time()
print("total time used: {}".format(end - start))

### LODA

In [ ]:
from pyod.models.loda import LODA

def loda(df, contamination=0.01, n_bins=10, n_random_cuts=100, cut_off=15, plot=True):
    
    x_ = df.run_time.values.reshape(-1, 1)
    highest, high, low = x_.max(), np.quantile(x_, q=0.99), np.quantile(x_, q=0.25)
    if df.run_time.std() == 0:
        out, normal = pd.DataFrame({'run_time':[]}), df
        print("Data has no variation, No outlier !")

    elif (highest - low) / low <= cut_off:
        out, normal = pd.DataFrame({'run_time':[]}), df
        print("cutoff rule applied with cutoff={}".format(cut_off))

    else:  
        if (high - low)/low < 0.5: # if majority of the data is narrowly distributed, lower the outlier ratio
            contamination = 0.1 * contamination
            print('contamination lowered with c={}'.format(contamination))
        clf = LODA(n_bins = n_bins, n_random_cuts=n_random_cuts, contamination = contamination) 
        clf.fit(x_)
        result = clf.predict(x_)
        tmp = pd.DataFrame({'anomaly_score': result, 'run_time': x_.reshape(-1)})
        out = tmp[(tmp.anomaly_score > 0)]
        normal = tmp[(tmp.anomaly_score == 0)]
        
    if plot:
        plot_result(out, normal, 'LODA')
    return out, normal

In [ ]:
start = time.time()

for i in range(10): # test 10 random samples
    seed = 78*i+131
    np.random.seed()
    j = np.random.randint(len(subset))
    func_name = subset.iloc[j].func_name
    sample = sample_portion(run_time, func_name, frac=0.55)
    print("seed = {}, func_name:{}".format(seed, func_name))
    loda(sample, plot=True);

end = time.time()
print("total time used: {}".format(end - start))

### LOCI

Memory leaks

In [ ]:
from pyod.models.loci import LOCI

In [ ]:
x_ = sample.run_time.values.reshape(-1, 1)

In [ ]:
loci = LOCI(contamination=0.01, alpha=0.5, k=3)
loci.fit(x_)
result = loc.predict(x_)

### LSCP

Locally Selective Combination of Parallel Outlier Ensembles (LSCP).

In [ ]:
from pyod.models.lscp import LSCP

In [ ]:
# detector_list = [LODA(), HBOS(), COPOD()]
detector_list = [LODA(), HBOS()]
lscp = LSCP(detector_list, local_region_size=30, local_max_features=0.5, n_bins=10, contamination=0.01)

In [ ]:
lscp.fit(x_[:3000])

In [ ]:
result = lscp.predict(x_)

### COF

It suffers from memory leaks

In [ ]:
from pyod.models.cof import COF


def cof(df, n_neighbors=20, contamination=0.01, cut_off = 20, plot=True):    
    x_ = df.run_time.values.reshape(-1, 1)
    if (x_.max() - x_.min()) / x_.min() <= cut_off:
        out, normal = None, df
    else:    
        clf = COF(n_neighbors = n_neighbors, contamination = contamination) 
        clf.fit(x_)
        result = clf.predict(x_)
        tmp = pd.DataFrame(np.stack([result, x_.reshape(-1)], axis=1), columns={'run_time', 'anomaly_score'})
        out = tmp[(tmp.anomaly_score > 0)]
        normal = tmp[(tmp.anomaly_score == 0)]
    if plot:
        plot_result(out, normal, 'cof')
    return out, normal

In [ ]:
start = time.time()
for i in range(10): # test 10 random samples
    np.random.seed(8*i+1)
    j = np.random.randint(len(subset))
    func_name = subset.iloc[j].func_name
    sample = sample_portion(run_time, func_name)
    cof(sample, plot=True);
end = time.time()
print("total time used: {}".format(end - start))